In [1]:
from weaPoints import WeaPoints, WindDataType, CalculationMethod
import numpy as np
import xarray
import pandas

# Loading Points

In [2]:
size = 10
lon = np.around(np.random.uniform(6.0, 14.0, size=size),5)
lat = np.around(np.random.uniform(48.0, 55.0, size=size),5)
lat_lon_coor = np.stack((lat, lon), axis=1)
level = np.around(np.random.uniform(120.0, 120.0, size=size),1)

testPoints = WeaPoints(
    lat_lon_coor = lat_lon_coor,
    level = level,
    wea_types = ["test_wea"] * size,
    #interpolation_method = [InterpolationMethod.LINEAR] * size,
    _interpolated_power_curves = True,
)

testPoints.point_list[:5]

[_WeaPoint(lat_lon_coor=array([54.881  , 12.39046]), level=120.0, wea_type='test_wea', x_y_coor=[105907.53413578896, 444285.49994777574]),
 _WeaPoint(lat_lon_coor=array([51.29095, 12.63768]), level=120.0, wea_type='test_wea', x_y_coor=[132022.7588105944, 45886.09314962475]),
 _WeaPoint(lat_lon_coor=array([48.11969, 13.94849]), level=120.0, wea_type='test_wea', x_y_coor=[238328.79786555542, -302848.49658686895]),
 _WeaPoint(lat_lon_coor=array([48.28712,  6.33321]), level=120.0, wea_type='test_wea', x_y_coor=[-325432.84460632544, -279708.42832648003]),
 _WeaPoint(lat_lon_coor=array([49.93187, 10.36485]), level=120.0, wea_type='test_wea', x_y_coor=[-26529.572444550355, -106650.14107252848])]

# Get Time series

### 15 min TSnetCDF

In [ ]:
testPoints.get_windpower_out(wind_data_type=WindDataType.TSNETCDF, time_frame=[2009,"2018-12-31"])

Eout = {}
for num, point in enumerate(testPoints.point_list):
    Eout[f"wea_{num+1}: {point.lat_lon_coor}"] = point.power_time_series

df = pandas.DataFrame(data=Eout,index=testPoints.time_periode)

df.resample("1m").sum().plot(figsize=(22,6))

In [ ]:
df.sum(axis=0).plot.bar(figsize=(22,6))

### 90m Mean

In [ ]:
testPoints.get_windpower_out(wind_data_type=WindDataType.MEAN90M)
Eout = {}
for num, point in enumerate(testPoints.point_list):
    Eout[f"wea_{num+1}: {point.lat_lon_coor}"] = point.power_time_series

df_90mean = pandas.DataFrame(data=Eout,index=testPoints.time_periode)
df_90mean.head()#.resample("14d").mean().plot(figsize=(22,6))

df_90mean.transpose().plot.bar(figsize=(22,6))

### 3km Mean 10a

In [3]:
testPoints.get_windpower_out(wind_data_type=WindDataType.MEAN3KM, calculation_method=CalculationMethod.WEIBULL)
Eout = {}
for num, point in enumerate(testPoints.point_list):
    Eout[f"wea_{num+1}: {point.lat_lon_coor}"] = point.power_time_series

df_90mean = pandas.DataFrame(data=Eout,index=testPoints.time_periode)
df_90mean.head()#.resample("14d").mean().plot(figsize=(22,6))

df_90mean.transpose().plot.bar(figsize=(22,6))

Passed time_frame valid.
mean3km10a data loaded.
rho


/opt/anaconda/anaconda3/lib/python3.8/site-packages/xarray/core/missing.py:563: RuntimeWarning: All-NaN slice encountered
  minval = np.nanmin(new_x.values)
/opt/anaconda/anaconda3/lib/python3.8/site-packages/xarray/core/missing.py:564: RuntimeWarning: All-NaN slice encountered
  maxval = np.nanmax(new_x.values)


KeyError: nan

### Comparison

In [ ]:
from datetime import datetime

comp = df_90mean.transpose()
comp["15min"] = df.sum(axis=0)
comp.columns = [i.strftime('%Y-%m-%d') if isinstance(i, datetime) else i for i in comp.columns]
comp = comp.rename(columns={comp.columns[0]:"wiebull_90m"})
comp = comp / 1000000000 # Wh zu TWh umrechnen
comp["diff"] = comp["15min"] - comp["wiebull_90m"]

ax = comp["diff"].plot.bar(figsize=(18,5))
ax.set_ylabel("10-Jahresertrag in TWh")

In [ ]:
import geopandas
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as cx

In [ ]:
points = [point.lat_lon_coor for point in testPoints.point_list]
lat = [point[0] for point in points]
lon = [point[1] for point in points]

geodf = geopandas.GeoDataFrame(
    comp, geometry=geopandas.points_from_xy(lon, lat))

geodf["y"] = geodf.geometry.y
geodf["x"] = geodf.geometry.x

geodf = geodf.sort_values(by='y', ascending=False)

fig, ax = plt.subplots(1, 2, figsize=(15,10))
fig.suptitle("Differenz der Energieerträge von 150 gleichen, zufällig plazierten Beispiel-Windkraftanlagen mit 120m Nabenhöhe \n zwischen 15min-3km-Daten und 10a-90m-Daten von 2009 bis 2018 in Terrawattstunden", fontsize=16)
# SCATTER
geodf.plot(kind="scatter", x="diff", y="y", ax=ax[0], c=geodf["diff"], cmap='plasma')
ax[0].set_ylabel("Breitengrad")
ax[0].set_xlabel("Differenz Energieertrag [TWh]")

# MAP
geodf.crs = "EPSG:4326"
geodf = geodf.to_crs(epsg=3857)

geodf.plot(column='diff', ax=ax[1], cmap='plasma')#, cax=cax)
cx.add_basemap(ax[1], source=cx.providers.CartoDB.Voyager)#, crs=geodf.crs)
ax[1].set_xticks([])
ax[1].set_yticks([])

#fig.savefig('Diff150Anlagen.eps', format='eps')

### 3km man

In [ ]:
path = "/uba/anemos_winddata/20191029_anemosDataFull/UBA-Windatlas/Statistics/10-Jahresmittel/histos.120m.2009-2018.nc"


xarray.open_dataset(path).interp(x=12.3,y=30.34).wbk

In [ ]:
path = "/uba/anemos_winddata/20191029_anemosDataFull/UBA-Windatlas/3arcsecs/D-3km.E5.3arcsecs.wbk.2009-2018.nc"

xarray.open_dataset(path, engine="h5netcdf").interp(x=6.27371826,y=47.94883789, level=120).wbk

In [ ]:
from enum import Enum, unique

@unique
class test (Enum):
    TEST1 = 1
    TEST2 = 2
    TEST3 = 3


In [ ]:
test = test.TEST1

if test is test.TEST1:
    print("jup")

if test is (test.TEST2 or test.TEST3):
    print("noup")